In [ ]:
"""
LSTM for Speech Emotion Recognition on IEMOCAP
Loss Function: Cross Entropy
"""

# ============== CELL 1: Installation ==============
# !pip install kagglehub librosa soundfile matplotlib tqdm numpy pandas scikit-learn

# ============== CELL 2: Download & Explore Dataset ==============
import kagglehub
import os

print("Downloading IEMOCAP dataset...")
dataset_path = kagglehub.dataset_download("dejolilandry/iemocapfullrelease")
print(f"Dataset path: {dataset_path}")

# Explore structure
print("\nExploring dataset structure...")
for item in os.listdir(dataset_path)[:10]:
    print(f"  {item}")

# Find wav files
wav_files = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.endswith('.wav'):
            wav_files.append(os.path.join(root, f))
print(f"\nTotal .wav files: {len(wav_files)}")
if wav_files:
    print("Sample paths:")
    for p in wav_files[:3]:
        print(f"  {p}")

# ============== CELL 3: Imports ==============
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import warnings
import re
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============== CELL 4: Configuration ==============
class Config:
    SAMPLE_RATE = 16000
    MAX_DURATION = 20  # seconds
    N_MFCC = 13  # Number of MFCC coefficients
    MAX_LENGTH = 128  # Max time steps for LSTM
    BATCH_SIZE = 32
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.001
    NUM_CLASSES = 4  # angry, happy, sad, neutral
    DEVICE = device
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    DROPOUT = 0.3
    BIDIRECTIONAL = True

config = Config()

# ============== CELL 5: Data Loading ==============
def load_iemocap_data(dataset_path):
    """
    Load IEMOCAP by parsing EmoEvaluation annotation files.
    IEMOCAP stores emotions in separate .txt files, not in wav filenames.
    """
    files, labels = [], []
    emotion_map = {'ang': 0, 'hap': 1, 'exc': 1, 'sad': 2, 'neu': 3}

    # Dictionary to store utterance_id -> emotion mapping
    utterance_emotions = {}

    # Find and parse all EmoEvaluation files
    print("Parsing emotion annotation files...")
    eval_files_found = 0

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            # Look for evaluation files in EmoEvaluation folders
            if f.endswith('.txt') and 'EmoEvaluation' in root:
                eval_files_found += 1
                eval_path = os.path.join(root, f)

                try:
                    with open(eval_path, 'r', errors='ignore') as ef:
                        for line in ef:
                            # Pattern: [start - end] utterance_id emotion [v, a, d]
                            match = re.match(
                                r'\[[\d\.]+ - [\d\.]+\]\s+(\S+)\s+(ang|hap|exc|sad|neu|fru|sur|fea|dis|xxx|oth)',
                                line.strip()
                            )
                            if match:
                                utt_id = match.group(1)
                                emotion = match.group(2).lower()
                                if emotion in emotion_map:
                                    utterance_emotions[utt_id] = emotion_map[emotion]
                except Exception as e:
                    continue

    print(f"Found {eval_files_found} evaluation files")
    print(f"Parsed {len(utterance_emotions)} utterance labels")

    # Now find corresponding wav files
    print("Matching wav files with labels...")
    wav_files_dict = {}

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            if f.endswith('.wav'):
                # Get utterance ID (filename without extension)
                utt_id = f.replace('.wav', '')
                wav_files_dict[utt_id] = os.path.join(root, f)

    # Match wav files with emotions
    for utt_id, emotion_label in utterance_emotions.items():
        if utt_id in wav_files_dict:
            files.append(wav_files_dict[utt_id])
            labels.append(emotion_label)

    print(f"Found {len(files)} labeled audio files")

    # Class distribution
    if files:
        unique, counts = np.unique(labels, return_counts=True)
        names = ['angry', 'happy', 'sad', 'neutral']
        print("\nClass distribution:")
        for u, c in zip(unique, counts):
            print(f"  {names[u]}: {c}")

    return files, labels

audio_files, labels = load_iemocap_data(dataset_path)

if len(audio_files) == 0:
    raise ValueError("No labeled files found! Check dataset structure.")

# ============== CELL 6: Dataset Class with MFCC Feature Extraction ==============
class IEMOCAPLSTMDataset(Dataset):
    def __init__(self, audio_files, labels, config):
        self.audio_files = audio_files
        self.labels = labels
        self.config = config
        self.max_len = config.SAMPLE_RATE * config.MAX_DURATION

    def __len__(self):
        return len(self.audio_files)

    def extract_features(self, audio, sr):
        """Extract MFCC features with deltas and delta-deltas"""
        # Extract MFCCs (13 coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=self.config.N_MFCC)

        # Extract delta and delta-delta
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

        # Concatenate features (13 + 13 + 13 = 39 features)
        features = np.concatenate([mfccs, mfcc_delta, mfcc_delta2], axis=0)

        return features.T  # Shape: (time_steps, features)

    def __getitem__(self, idx):
        path = self.audio_files[idx]
        label = self.labels[idx]

        try:
            # Load audio
            audio, sr = librosa.load(path, sr=self.config.SAMPLE_RATE, mono=True)
            audio = librosa.util.normalize(audio)

            # Pad/truncate to fixed length
            if len(audio) > self.max_len:
                audio = audio[:self.max_len]
            else:
                audio = np.pad(audio, (0, self.max_len - len(audio)))

            # Extract MFCC features
            features = self.extract_features(audio, sr)

            # Pad or truncate to fixed length
            if features.shape[0] < self.config.MAX_LENGTH:
                # Pad with zeros
                padding = np.zeros((self.config.MAX_LENGTH - features.shape[0], features.shape[1]))
                features = np.vstack([features, padding])
                seq_length = features.shape[0] - padding.shape[0]
            else:
                # Truncate
                features = features[:self.config.MAX_LENGTH]
                seq_length = self.config.MAX_LENGTH

            return torch.tensor(features, dtype=torch.float32), label, seq_length

        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return zeros if error
            return torch.zeros(self.config.MAX_LENGTH, self.config.N_MFCC * 3), label, 0

# ============== CELL 7: Create DataLoaders ==============
X_train, X_test, y_train, y_test = train_test_split(
    audio_files, labels, test_size=0.2, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)

train_dataset = IEMOCAPLSTMDataset(X_train, y_train, config)
val_dataset = IEMOCAPLSTMDataset(X_val, y_val, config)
test_dataset = IEMOCAPLSTMDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
                        shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                         shuffle=False, num_workers=2)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# ============== CELL 8: LSTM Model ==============
class EmotionLSTM(nn.Module):
    """Bidirectional LSTM for Speech Emotion Recognition"""

    def __init__(self, config):
        super(EmotionLSTM, self).__init__()

        self.hidden_size = config.HIDDEN_SIZE
        self.num_layers = config.NUM_LAYERS
        self.bidirectional = config.BIDIRECTIONAL

        input_size = config.N_MFCC * 3  # 13 MFCCs + 13 deltas + 13 delta-deltas = 39

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT if config.NUM_LAYERS > 1 else 0,
            bidirectional=config.BIDIRECTIONAL
        )

        # Fully connected layers
        lstm_output_size = config.HIDDEN_SIZE * 2 if config.BIDIRECTIONAL else config.HIDDEN_SIZE
        self.fc1 = nn.Linear(lstm_output_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(config.DROPOUT)
        self.fc2 = nn.Linear(64, config.NUM_CLASSES)
        self.relu = nn.ReLU()

    def forward(self, x, seq_lengths=None):
        # x shape: (batch, seq_len, input_size)

        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Use the last output (for bidirectional, concatenate both directions)
        if self.bidirectional:
            # Concatenate the final forward and backward hidden states
            last_output = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            last_output = hidden[-1]

        # Fully connected layers
        out = self.fc1(last_output)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return out

# ============== CELL 9: Training Function ==============
def train_lstm(model, train_loader, val_loader, config):
    """Train LSTM with Cross Entropy Loss"""

    print(f"\n{'='*60}")
    print(f"Training Bidirectional LSTM for Emotion Recognition")
    print(f"{'='*60}")

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.NUM_EPOCHS)

    best_val_acc = 0
    best_model_state = None
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    patience = 10
    patience_counter = 0

    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]')
        for features, labels, seq_lengths in train_bar:
            features = features.to(config.DEVICE)
            labels = labels.to(config.DEVICE).long()

            # Forward pass
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # Statistics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            train_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for features, labels, seq_lengths in val_loader:
                features = features.to(config.DEVICE)
                labels = labels.to(config.DEVICE).long()

                outputs = model(features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        # Store metrics
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        scheduler.step()

        # Early stopping
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            print('  ✅ Best model saved!')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'\n⚠️ Early stopping triggered after {epoch+1} epochs')
                break

    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    return model, history, best_val_acc

# ============== CELL 10: Evaluation Function ==============
def evaluate_model(model, test_loader, config):
    """Evaluate model on test set"""
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for features, labels, seq_lengths in test_loader:
            features = features.to(config.DEVICE)
            outputs = model(features)
            preds.extend(outputs.argmax(1).cpu().numpy())
            true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')
    cm = confusion_matrix(true_labels, preds)

    return acc, f1, cm, preds, true_labels

# ============== CELL 11: Train and Evaluate ==============
print(f"\n{'='*60}")
print("Initializing LSTM Model")
print(f"{'='*60}")

model = EmotionLSTM(config).to(config.DEVICE)

print(f"\n📊 Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train
model, history, best_val_acc = train_lstm(model, train_loader, val_loader, config)

# Evaluate on test set
print(f"\n{'='*60}")
print("Evaluating on Test Set")
print(f"{'='*60}")

test_acc, test_f1, cm, preds, true_labels = evaluate_model(model, test_loader, config)

print(f"\n🏆 FINAL RESULTS:")
print(f"  Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"  Test Accuracy: {test_acc*100:.2f}%")
print(f"  Test F1 Score: {test_f1:.4f}")

# Classification report
emotion_names = ['Angry', 'Happy', 'Sad', 'Neutral']
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, preds, target_names=emotion_names))

# ============== CELL 12: Visualization ==============
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Training Loss
ax1 = axes[0, 0]
ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Training Accuracy
ax2 = axes[0, 1]
ax2.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: Confusion Matrix
ax3 = axes[1, 0]
im = ax3.imshow(cm, cmap='Blues', aspect='auto')
ax3.set_xticks(range(4))
ax3.set_yticks(range(4))
ax3.set_xticklabels(emotion_names, rotation=45)
ax3.set_yticklabels(emotion_names)
ax3.set_xlabel('Predicted', fontsize=12)
ax3.set_ylabel('True', fontsize=12)
ax3.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
for i in range(4):
    for j in range(4):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax3.text(j, i, cm[i, j], ha='center', va='center',
                fontsize=14, color=text_color, fontweight='bold')
plt.colorbar(im, ax=ax3)

# Plot 4: Per-class Accuracy
ax4 = axes[1, 1]
per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
bars = ax4.bar(emotion_names, per_class_acc, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'])
ax4.set_ylabel('Accuracy (%)', fontsize=12)
ax4.set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
ax4.set_ylim([0, 100])
for bar, val in zip(bars, per_class_acc):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.1f}%', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('lstm_emotion_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Results saved to 'lstm_emotion_results.png'")

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'history': history
}, 'lstm_emotion_model.pth')

print("✅ Model saved to 'lstm_emotion_model.pth'")

In [ ]:
from google.colab import files
files.download('lstm_emotion_model.pth')
print("✅ Model downloaded as 'lstm_emotion_model.pth'")

In [ ]:
"""
LSTM for Speech Emotion Recognition on IEMOCAP
Loss Function: Label Smoothing Cross Entropy
"""

# ============== CELL 1: Installation ==============
# !pip install kagglehub librosa soundfile matplotlib tqdm numpy pandas scikit-learn

# ============== CELL 2: Download & Explore Dataset ==============
import kagglehub
import os

print("Downloading IEMOCAP dataset...")
dataset_path = kagglehub.dataset_download("dejolilandry/iemocapfullrelease")
print(f"Dataset path: {dataset_path}")

# Explore structure
print("\nExploring dataset structure...")
for item in os.listdir(dataset_path)[:10]:
    print(f"  {item}")

# Find wav files
wav_files = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.endswith('.wav'):
            wav_files.append(os.path.join(root, f))
print(f"\nTotal .wav files: {len(wav_files)}")
if wav_files:
    print("Sample paths:")
    for p in wav_files[:3]:
        print(f"  {p}")

# ============== CELL 3: Imports ==============
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import warnings
import re
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============== CELL 4: Configuration ==============
class Config:
    SAMPLE_RATE = 16000
    MAX_DURATION = 20  # seconds
    N_MFCC = 13  # Number of MFCC coefficients
    MAX_LENGTH = 128  # Max time steps for LSTM
    BATCH_SIZE = 32
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.001
    NUM_CLASSES = 4  # angry, happy, sad, neutral
    DEVICE = device
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    DROPOUT = 0.3
    BIDIRECTIONAL = True
    LABEL_SMOOTHING = 0.1  # Label smoothing factor (0.0 = no smoothing, 0.1 is common)

config = Config()

# ============== CELL 5: Data Loading ==============
def load_iemocap_data(dataset_path):
    """
    Load IEMOCAP by parsing EmoEvaluation annotation files.
    IEMOCAP stores emotions in separate .txt files, not in wav filenames.
    """
    files, labels = [], []
    emotion_map = {'ang': 0, 'hap': 1, 'exc': 1, 'sad': 2, 'neu': 3}

    # Dictionary to store utterance_id -> emotion mapping
    utterance_emotions = {}

    # Find and parse all EmoEvaluation files
    print("Parsing emotion annotation files...")
    eval_files_found = 0

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            # Look for evaluation files in EmoEvaluation folders
            if f.endswith('.txt') and 'EmoEvaluation' in root:
                eval_files_found += 1
                eval_path = os.path.join(root, f)

                try:
                    with open(eval_path, 'r', errors='ignore') as ef:
                        for line in ef:
                            # Pattern: [start - end] utterance_id emotion [v, a, d]
                            match = re.match(
                                r'\[[\d\.]+ - [\d\.]+\]\s+(\S+)\s+(ang|hap|exc|sad|neu|fru|sur|fea|dis|xxx|oth)',
                                line.strip()
                            )
                            if match:
                                utt_id = match.group(1)
                                emotion = match.group(2).lower()
                                if emotion in emotion_map:
                                    utterance_emotions[utt_id] = emotion_map[emotion]
                except Exception as e:
                    continue

    print(f"Found {eval_files_found} evaluation files")
    print(f"Parsed {len(utterance_emotions)} utterance labels")

    # Now find corresponding wav files
    print("Matching wav files with labels...")
    wav_files_dict = {}

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            if f.endswith('.wav'):
                # Get utterance ID (filename without extension)
                utt_id = f.replace('.wav', '')
                wav_files_dict[utt_id] = os.path.join(root, f)

    # Match wav files with emotions
    for utt_id, emotion_label in utterance_emotions.items():
        if utt_id in wav_files_dict:
            files.append(wav_files_dict[utt_id])
            labels.append(emotion_label)

    print(f"Found {len(files)} labeled audio files")

    # Class distribution
    if files:
        unique, counts = np.unique(labels, return_counts=True)
        names = ['angry', 'happy', 'sad', 'neutral']
        print("\nClass distribution:")
        for u, c in zip(unique, counts):
            print(f"  {names[u]}: {c}")

    return files, labels

audio_files, labels = load_iemocap_data(dataset_path)

if len(audio_files) == 0:
    raise ValueError("No labeled files found! Check dataset structure.")

# ============== CELL 6: Dataset Class with MFCC Feature Extraction ==============
class IEMOCAPLSTMDataset(Dataset):
    def __init__(self, audio_files, labels, config):
        self.audio_files = audio_files
        self.labels = labels
        self.config = config
        self.max_len = config.SAMPLE_RATE * config.MAX_DURATION

    def __len__(self):
        return len(self.audio_files)

    def extract_features(self, audio, sr):
        """Extract MFCC features with deltas and delta-deltas"""
        # Extract MFCCs (13 coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=self.config.N_MFCC)

        # Extract delta and delta-delta
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

        # Concatenate features (13 + 13 + 13 = 39 features)
        features = np.concatenate([mfccs, mfcc_delta, mfcc_delta2], axis=0)

        return features.T  # Shape: (time_steps, features)

    def __getitem__(self, idx):
        path = self.audio_files[idx]
        label = self.labels[idx]

        try:
            # Load audio
            audio, sr = librosa.load(path, sr=self.config.SAMPLE_RATE, mono=True)
            audio = librosa.util.normalize(audio)

            # Pad/truncate to fixed length
            if len(audio) > self.max_len:
                audio = audio[:self.max_len]
            else:
                audio = np.pad(audio, (0, self.max_len - len(audio)))

            # Extract MFCC features
            features = self.extract_features(audio, sr)

            # Pad or truncate to fixed length
            if features.shape[0] < self.config.MAX_LENGTH:
                # Pad with zeros
                padding = np.zeros((self.config.MAX_LENGTH - features.shape[0], features.shape[1]))
                features = np.vstack([features, padding])
                seq_length = features.shape[0] - padding.shape[0]
            else:
                # Truncate
                features = features[:self.config.MAX_LENGTH]
                seq_length = self.config.MAX_LENGTH

            return torch.tensor(features, dtype=torch.float32), label, seq_length

        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return zeros if error
            return torch.zeros(self.config.MAX_LENGTH, self.config.N_MFCC * 3), label, 0

# ============== CELL 7: Create DataLoaders ==============
X_train, X_test, y_train, y_test = train_test_split(
    audio_files, labels, test_size=0.2, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)

train_dataset = IEMOCAPLSTMDataset(X_train, y_train, config)
val_dataset = IEMOCAPLSTMDataset(X_val, y_val, config)
test_dataset = IEMOCAPLSTMDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
                        shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                         shuffle=False, num_workers=2)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# ============== CELL 8: Label Smoothing Cross Entropy Loss ==============
class LabelSmoothingCrossEntropy(nn.Module):
    """
    Label Smoothing Cross Entropy Loss

    Instead of using hard targets (e.g., [0, 1, 0, 0]), label smoothing creates
    soft targets (e.g., [0.033, 0.9, 0.033, 0.033]) to prevent overconfidence.

    Args:
        smoothing: smoothing factor (typically 0.1)
        reduction: 'mean', 'sum', or 'none'
    """
    def __init__(self, smoothing=0.1, reduction='mean'):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.smoothing = smoothing
        self.reduction = reduction

    def forward(self, pred, target):
        """
        Args:
            pred: predictions (batch_size, num_classes) - logits (not softmax)
            target: ground truth labels (batch_size,) - class indices
        """
        num_classes = pred.size(-1)
        log_probs = F.log_softmax(pred, dim=-1)

        # Create smoothed labels
        # True class gets (1 - smoothing) probability
        # Other classes share smoothing probability equally
        with torch.no_grad():
            true_dist = torch.zeros_like(log_probs)
            true_dist.fill_(self.smoothing / (num_classes - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.smoothing)

        # Compute KL divergence between predicted and smoothed distributions
        loss = torch.sum(-true_dist * log_probs, dim=-1)

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# ============== CELL 9: LSTM Model ==============
class EmotionLSTM(nn.Module):
    """Bidirectional LSTM for Speech Emotion Recognition"""

    def __init__(self, config):
        super(EmotionLSTM, self).__init__()

        self.hidden_size = config.HIDDEN_SIZE
        self.num_layers = config.NUM_LAYERS
        self.bidirectional = config.BIDIRECTIONAL

        input_size = config.N_MFCC * 3  # 13 MFCCs + 13 deltas + 13 delta-deltas = 39

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT if config.NUM_LAYERS > 1 else 0,
            bidirectional=config.BIDIRECTIONAL
        )

        # Fully connected layers
        lstm_output_size = config.HIDDEN_SIZE * 2 if config.BIDIRECTIONAL else config.HIDDEN_SIZE
        self.fc1 = nn.Linear(lstm_output_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(config.DROPOUT)
        self.fc2 = nn.Linear(64, config.NUM_CLASSES)
        self.relu = nn.ReLU()

    def forward(self, x, seq_lengths=None):
        # x shape: (batch, seq_len, input_size)

        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Use the last output (for bidirectional, concatenate both directions)
        if self.bidirectional:
            # Concatenate the final forward and backward hidden states
            last_output = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            last_output = hidden[-1]

        # Fully connected layers
        out = self.fc1(last_output)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return out

# ============== CELL 10: Training Function ==============
def train_lstm(model, train_loader, val_loader, config):
    """Train LSTM with Label Smoothing Cross Entropy Loss"""

    print(f"\n{'='*60}")
    print(f"Training Bidirectional LSTM for Emotion Recognition")
    print(f"Loss Function: Label Smoothing Cross Entropy (α={config.LABEL_SMOOTHING})")
    print(f"{'='*60}")

    # Use Label Smoothing Cross Entropy
    criterion = LabelSmoothingCrossEntropy(smoothing=config.LABEL_SMOOTHING)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.NUM_EPOCHS)

    best_val_acc = 0
    best_model_state = None
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    patience = 10
    patience_counter = 0

    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]')
        for features, labels, seq_lengths in train_bar:
            features = features.to(config.DEVICE)
            labels = labels.to(config.DEVICE).long()

            # Forward pass
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # Statistics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            train_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for features, labels, seq_lengths in val_loader:
                features = features.to(config.DEVICE)
                labels = labels.to(config.DEVICE).long()

                outputs = model(features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        # Store metrics
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        scheduler.step()

        # Early stopping
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            print('  ✅ Best model saved!')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'\n⚠️ Early stopping triggered after {epoch+1} epochs')
                break

    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    return model, history, best_val_acc

# ============== CELL 11: Evaluation Function ==============
def evaluate_model(model, test_loader, config):
    """Evaluate model on test set"""
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for features, labels, seq_lengths in test_loader:
            features = features.to(config.DEVICE)
            outputs = model(features)
            preds.extend(outputs.argmax(1).cpu().numpy())
            true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')
    cm = confusion_matrix(true_labels, preds)

    return acc, f1, cm, preds, true_labels

# ============== CELL 12: Train and Evaluate ==============
print(f"\n{'='*60}")
print("Initializing LSTM Model")
print(f"{'='*60}")

model = EmotionLSTM(config).to(config.DEVICE)

print(f"\n📊 Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n🔧 Loss Function: Label Smoothing Cross Entropy")
print(f"   Smoothing Factor: {config.LABEL_SMOOTHING}")

# Train
model, history, best_val_acc = train_lstm(model, train_loader, val_loader, config)

# Evaluate on test set
print(f"\n{'='*60}")
print("Evaluating on Test Set")
print(f"{'='*60}")

test_acc, test_f1, cm, preds, true_labels = evaluate_model(model, test_loader, config)

print(f"\n🏆 FINAL RESULTS:")
print(f"  Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"  Test Accuracy: {test_acc*100:.2f}%")
print(f"  Test F1 Score: {test_f1:.4f}")

# Classification report
emotion_names = ['Angry', 'Happy', 'Sad', 'Neutral']
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, preds, target_names=emotion_names))

# ============== CELL 13: Visualization ==============
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Training Loss
ax1 = axes[0, 0]
ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training and Validation Loss\n(Label Smoothing CE)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Training Accuracy
ax2 = axes[0, 1]
ax2.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('Training and Validation Accuracy\n(Label Smoothing CE)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: Confusion Matrix
ax3 = axes[1, 0]
im = ax3.imshow(cm, cmap='Blues', aspect='auto')
ax3.set_xticks(range(4))
ax3.set_yticks(range(4))
ax3.set_xticklabels(emotion_names, rotation=45)
ax3.set_yticklabels(emotion_names)
ax3.set_xlabel('Predicted', fontsize=12)
ax3.set_ylabel('True', fontsize=12)
ax3.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
for i in range(4):
    for j in range(4):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax3.text(j, i, cm[i, j], ha='center', va='center',
                fontsize=14, color=text_color, fontweight='bold')
plt.colorbar(im, ax=ax3)

# Plot 4: Per-class Accuracy
ax4 = axes[1, 1]
per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
bars = ax4.bar(emotion_names, per_class_acc, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'])
ax4.set_ylabel('Accuracy (%)', fontsize=12)
ax4.set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
ax4.set_ylim([0, 100])
for bar, val in zip(bars, per_class_acc):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.1f}%', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('lstm_emotion_results_label_smoothing.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Results saved to 'lstm_emotion_results_label_smoothing.png'")

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'history': history,
    'loss_function': 'Label Smoothing Cross Entropy',
    'smoothing_factor': config.LABEL_SMOOTHING
}, 'lstm_emotion_model_label_smoothing.pth')

print("✅ Model saved to 'lstm_emotion_model_label_smoothing.pth'")

In [ ]:
from google.colab import files
files.download('lstm_emotion_model_label_smoothing.pth')
print("✅ Model downloaded as 'lstm_emotion_model_label_smoothing.pth'")

In [ ]:
"""
LSTM for Speech Emotion Recognition on IEMOCAP
Loss Function: Focal Loss
"""

# ============== CELL 1: Installation ==============
# !pip install kagglehub librosa soundfile matplotlib tqdm numpy pandas scikit-learn

# ============== CELL 2: Download & Explore Dataset ==============
import kagglehub
import os

print("Downloading IEMOCAP dataset...")
dataset_path = kagglehub.dataset_download("dejolilandry/iemocapfullrelease")
print(f"Dataset path: {dataset_path}")

# Explore structure
print("\nExploring dataset structure...")
for item in os.listdir(dataset_path)[:10]:
    print(f"  {item}")

# Find wav files
wav_files = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.endswith('.wav'):
            wav_files.append(os.path.join(root, f))
print(f"\nTotal .wav files: {len(wav_files)}")
if wav_files:
    print("Sample paths:")
    for p in wav_files[:3]:
        print(f"  {p}")

# ============== CELL 3: Imports ==============
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import warnings
import re
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============== CELL 4: Configuration ==============
class Config:
    SAMPLE_RATE = 16000
    MAX_DURATION = 20  # seconds
    N_MFCC = 13  # Number of MFCC coefficients
    MAX_LENGTH = 128  # Max time steps for LSTM
    BATCH_SIZE = 32
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.001
    NUM_CLASSES = 4  # angry, happy, sad, neutral
    DEVICE = device
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    DROPOUT = 0.3
    BIDIRECTIONAL = True
    # Focal Loss parameters
    FOCAL_ALPHA = None  # Class weights (None for balanced, or list of weights)
    FOCAL_GAMMA = 2.0  # Focusing parameter (0 = CE, 2 is common, higher = more focus on hard examples)

config = Config()

# ============== CELL 5: Data Loading ==============
def load_iemocap_data(dataset_path):
    """
    Load IEMOCAP by parsing EmoEvaluation annotation files.
    IEMOCAP stores emotions in separate .txt files, not in wav filenames.
    """
    files, labels = [], []
    emotion_map = {'ang': 0, 'hap': 1, 'exc': 1, 'sad': 2, 'neu': 3}

    # Dictionary to store utterance_id -> emotion mapping
    utterance_emotions = {}

    # Find and parse all EmoEvaluation files
    print("Parsing emotion annotation files...")
    eval_files_found = 0

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            # Look for evaluation files in EmoEvaluation folders
            if f.endswith('.txt') and 'EmoEvaluation' in root:
                eval_files_found += 1
                eval_path = os.path.join(root, f)

                try:
                    with open(eval_path, 'r', errors='ignore') as ef:
                        for line in ef:
                            # Pattern: [start - end] utterance_id emotion [v, a, d]
                            match = re.match(
                                r'\[[\d\.]+ - [\d\.]+\]\s+(\S+)\s+(ang|hap|exc|sad|neu|fru|sur|fea|dis|xxx|oth)',
                                line.strip()
                            )
                            if match:
                                utt_id = match.group(1)
                                emotion = match.group(2).lower()
                                if emotion in emotion_map:
                                    utterance_emotions[utt_id] = emotion_map[emotion]
                except Exception as e:
                    continue

    print(f"Found {eval_files_found} evaluation files")
    print(f"Parsed {len(utterance_emotions)} utterance labels")

    # Now find corresponding wav files
    print("Matching wav files with labels...")
    wav_files_dict = {}

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            if f.endswith('.wav'):
                # Get utterance ID (filename without extension)
                utt_id = f.replace('.wav', '')
                wav_files_dict[utt_id] = os.path.join(root, f)

    # Match wav files with emotions
    for utt_id, emotion_label in utterance_emotions.items():
        if utt_id in wav_files_dict:
            files.append(wav_files_dict[utt_id])
            labels.append(emotion_label)

    print(f"Found {len(files)} labeled audio files")

    # Class distribution
    if files:
        unique, counts = np.unique(labels, return_counts=True)
        names = ['angry', 'happy', 'sad', 'neutral']
        print("\nClass distribution:")
        for u, c in zip(unique, counts):
            print(f"  {names[u]}: {c}")

    return files, labels

audio_files, labels = load_iemocap_data(dataset_path)

if len(audio_files) == 0:
    raise ValueError("No labeled files found! Check dataset structure.")

# ============== CELL 6: Dataset Class with MFCC Feature Extraction ==============
class IEMOCAPLSTMDataset(Dataset):
    def __init__(self, audio_files, labels, config):
        self.audio_files = audio_files
        self.labels = labels
        self.config = config
        self.max_len = config.SAMPLE_RATE * config.MAX_DURATION

    def __len__(self):
        return len(self.audio_files)

    def extract_features(self, audio, sr):
        """Extract MFCC features with deltas and delta-deltas"""
        # Extract MFCCs (13 coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=self.config.N_MFCC)

        # Extract delta and delta-delta
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

        # Concatenate features (13 + 13 + 13 = 39 features)
        features = np.concatenate([mfccs, mfcc_delta, mfcc_delta2], axis=0)

        return features.T  # Shape: (time_steps, features)

    def __getitem__(self, idx):
        path = self.audio_files[idx]
        label = self.labels[idx]

        try:
            # Load audio
            audio, sr = librosa.load(path, sr=self.config.SAMPLE_RATE, mono=True)
            audio = librosa.util.normalize(audio)

            # Pad/truncate to fixed length
            if len(audio) > self.max_len:
                audio = audio[:self.max_len]
            else:
                audio = np.pad(audio, (0, self.max_len - len(audio)))

            # Extract MFCC features
            features = self.extract_features(audio, sr)

            # Pad or truncate to fixed length
            if features.shape[0] < self.config.MAX_LENGTH:
                # Pad with zeros
                padding = np.zeros((self.config.MAX_LENGTH - features.shape[0], features.shape[1]))
                features = np.vstack([features, padding])
                seq_length = features.shape[0] - padding.shape[0]
            else:
                # Truncate
                features = features[:self.config.MAX_LENGTH]
                seq_length = self.config.MAX_LENGTH

            return torch.tensor(features, dtype=torch.float32), label, seq_length

        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return zeros if error
            return torch.zeros(self.config.MAX_LENGTH, self.config.N_MFCC * 3), label, 0

# ============== CELL 7: Create DataLoaders ==============
X_train, X_test, y_train, y_test = train_test_split(
    audio_files, labels, test_size=0.2, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)

train_dataset = IEMOCAPLSTMDataset(X_train, y_train, config)
val_dataset = IEMOCAPLSTMDataset(X_val, y_val, config)
test_dataset = IEMOCAPLSTMDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
                        shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                         shuffle=False, num_workers=2)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# ============== CELL 8: Focal Loss Implementation ==============
class FocalLoss(nn.Module):
    """
    Focal Loss for addressing class imbalance

    Focal Loss focuses training on hard examples by down-weighting easy examples.
    It adds a modulating factor (1 - p_t)^gamma to the standard cross entropy loss.

    Formula: FL(p_t) = -α_t * (1 - p_t)^γ * log(p_t)

    where:
    - p_t is the model's estimated probability for the true class
    - α_t is the class weight for class t (optional)
    - γ (gamma) is the focusing parameter:
        * γ = 0: Focal Loss = Cross Entropy
        * γ = 2: Common default, strongly focuses on hard examples
        * Higher γ: More focus on hard-to-classify examples

    Args:
        alpha: Class weights (None for no weighting, or tensor of shape [num_classes])
        gamma: Focusing parameter (default: 2.0)
        reduction: 'mean', 'sum', or 'none'
    """
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Args:
            inputs: predictions (batch_size, num_classes) - logits (not softmax)
            targets: ground truth labels (batch_size,) - class indices
        """
        # Compute cross entropy loss
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        # Get probability of true class
        p = torch.exp(-ce_loss)  # p_t in the paper

        # Compute focal loss
        focal_loss = (1 - p) ** self.gamma * ce_loss

        # Apply class weights if provided
        if self.alpha is not None:
            if isinstance(self.alpha, (list, np.ndarray)):
                alpha = torch.tensor(self.alpha).to(inputs.device)
            else:
                alpha = self.alpha

            # Get alpha for each sample based on target class
            alpha_t = alpha[targets]
            focal_loss = alpha_t * focal_loss

        # Apply reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# ============== CELL 9: LSTM Model ==============
class EmotionLSTM(nn.Module):
    """Bidirectional LSTM for Speech Emotion Recognition"""

    def __init__(self, config):
        super(EmotionLSTM, self).__init__()

        self.hidden_size = config.HIDDEN_SIZE
        self.num_layers = config.NUM_LAYERS
        self.bidirectional = config.BIDIRECTIONAL

        input_size = config.N_MFCC * 3  # 13 MFCCs + 13 deltas + 13 delta-deltas = 39

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT if config.NUM_LAYERS > 1 else 0,
            bidirectional=config.BIDIRECTIONAL
        )

        # Fully connected layers
        lstm_output_size = config.HIDDEN_SIZE * 2 if config.BIDIRECTIONAL else config.HIDDEN_SIZE
        self.fc1 = nn.Linear(lstm_output_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(config.DROPOUT)
        self.fc2 = nn.Linear(64, config.NUM_CLASSES)
        self.relu = nn.ReLU()

    def forward(self, x, seq_lengths=None):
        # x shape: (batch, seq_len, input_size)

        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Use the last output (for bidirectional, concatenate both directions)
        if self.bidirectional:
            # Concatenate the final forward and backward hidden states
            last_output = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            last_output = hidden[-1]

        # Fully connected layers
        out = self.fc1(last_output)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return out

# ============== CELL 10: Training Function ==============
def train_lstm(model, train_loader, val_loader, config):
    """Train LSTM with Focal Loss"""

    print(f"\n{'='*60}")
    print(f"Training Bidirectional LSTM for Emotion Recognition")
    print(f"Loss Function: Focal Loss (γ={config.FOCAL_GAMMA})")
    print(f"{'='*60}")

    # Use Focal Loss
    criterion = FocalLoss(alpha=config.FOCAL_ALPHA, gamma=config.FOCAL_GAMMA)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.NUM_EPOCHS)

    best_val_acc = 0
    best_model_state = None
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    patience = 10
    patience_counter = 0

    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]')
        for features, labels, seq_lengths in train_bar:
            features = features.to(config.DEVICE)
            labels = labels.to(config.DEVICE).long()

            # Forward pass
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # Statistics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            train_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for features, labels, seq_lengths in val_loader:
                features = features.to(config.DEVICE)
                labels = labels.to(config.DEVICE).long()

                outputs = model(features)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        # Store metrics
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        scheduler.step()

        # Early stopping
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            print('  ✅ Best model saved!')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'\n⚠️ Early stopping triggered after {epoch+1} epochs')
                break

    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    return model, history, best_val_acc

# ============== CELL 11: Evaluation Function ==============
def evaluate_model(model, test_loader, config):
    """Evaluate model on test set"""
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for features, labels, seq_lengths in test_loader:
            features = features.to(config.DEVICE)
            outputs = model(features)
            preds.extend(outputs.argmax(1).cpu().numpy())
            true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')
    cm = confusion_matrix(true_labels, preds)

    return acc, f1, cm, preds, true_labels

# ============== CELL 12: Train and Evaluate ==============
print(f"\n{'='*60}")
print("Initializing LSTM Model")
print(f"{'='*60}")

model = EmotionLSTM(config).to(config.DEVICE)

print(f"\n📊 Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n🔧 Loss Function: Focal Loss")
print(f"   Gamma (γ): {config.FOCAL_GAMMA}")
print(f"   Alpha (class weights): {config.FOCAL_ALPHA if config.FOCAL_ALPHA else 'None (balanced)'}")

# Train
model, history, best_val_acc = train_lstm(model, train_loader, val_loader, config)

# Evaluate on test set
print(f"\n{'='*60}")
print("Evaluating on Test Set")
print(f"{'='*60}")

test_acc, test_f1, cm, preds, true_labels = evaluate_model(model, test_loader, config)

print(f"\n🏆 FINAL RESULTS:")
print(f"  Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"  Test Accuracy: {test_acc*100:.2f}%")
print(f"  Test F1 Score: {test_f1:.4f}")

# Classification report
emotion_names = ['Angry', 'Happy', 'Sad', 'Neutral']
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, preds, target_names=emotion_names))

# ============== CELL 13: Visualization ==============
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Training Loss
ax1 = axes[0, 0]
ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title(f'Training and Validation Loss\n(Focal Loss, γ={config.FOCAL_GAMMA})',
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Training Accuracy
ax2 = axes[0, 1]
ax2.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title(f'Training and Validation Accuracy\n(Focal Loss, γ={config.FOCAL_GAMMA})',
              fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: Confusion Matrix
ax3 = axes[1, 0]
im = ax3.imshow(cm, cmap='Blues', aspect='auto')
ax3.set_xticks(range(4))
ax3.set_yticks(range(4))
ax3.set_xticklabels(emotion_names, rotation=45)
ax3.set_yticklabels(emotion_names)
ax3.set_xlabel('Predicted', fontsize=12)
ax3.set_ylabel('True', fontsize=12)
ax3.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
for i in range(4):
    for j in range(4):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax3.text(j, i, cm[i, j], ha='center', va='center',
                fontsize=14, color=text_color, fontweight='bold')
plt.colorbar(im, ax=ax3)

# Plot 4: Per-class Accuracy
ax4 = axes[1, 1]
per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
bars = ax4.bar(emotion_names, per_class_acc, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'])
ax4.set_ylabel('Accuracy (%)', fontsize=12)
ax4.set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
ax4.set_ylim([0, 100])
for bar, val in zip(bars, per_class_acc):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.1f}%', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('lstm_emotion_results_focal_loss.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Results saved to 'lstm_emotion_results_focal_loss.png'")

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'history': history,
    'loss_function': 'Focal Loss',
    'focal_gamma': config.FOCAL_GAMMA,
    'focal_alpha': config.FOCAL_ALPHA
}, 'lstm_emotion_model_focal_loss.pth')

print("✅ Model saved to 'lstm_emotion_model_focal_loss.pth'")

In [ ]:
from google.colab import files
files.download('lstm_emotion_model_focal_loss.pth')
print("✅ Model downloaded as 'lstm_emotion_model_focal_loss.pth'")

In [ ]:
"""
LSTM for Speech Emotion Recognition on IEMOCAP
Loss Function: ArcFace (Additive Angular Margin Loss)
"""

# ============== CELL 1: Installation ==============
# !pip install kagglehub librosa soundfile matplotlib tqdm numpy pandas scikit-learn

# ============== CELL 2: Download & Explore Dataset ==============
import kagglehub
import os

print("Downloading IEMOCAP dataset...")
dataset_path = kagglehub.dataset_download("dejolilandry/iemocapfullrelease")
print(f"Dataset path: {dataset_path}")

# Explore structure
print("\nExploring dataset structure...")
for item in os.listdir(dataset_path)[:10]:
    print(f"  {item}")

# Find wav files
wav_files = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.endswith('.wav'):
            wav_files.append(os.path.join(root, f))
print(f"\nTotal .wav files: {len(wav_files)}")
if wav_files:
    print("Sample paths:")
    for p in wav_files[:3]:
        print(f"  {p}")

# ============== CELL 3: Imports ==============
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import warnings
import re
import math
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============== CELL 4: Configuration ==============
class Config:
    SAMPLE_RATE = 16000
    MAX_DURATION = 20  # seconds
    N_MFCC = 13  # Number of MFCC coefficients
    MAX_LENGTH = 128  # Max time steps for LSTM
    BATCH_SIZE = 32
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.001
    NUM_CLASSES = 4  # angry, happy, sad, neutral
    DEVICE = device
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    DROPOUT = 0.3
    BIDIRECTIONAL = True
    # ArcFace parameters
    ARCFACE_S = 30.0  # Scale parameter (typically 30-64)
    ARCFACE_M = 0.50  # Angular margin (typically 0.3-0.5)
    EMBEDDING_SIZE = 128  # Size of the embedding before ArcFace layer

config = Config()

# ============== CELL 5: Data Loading ==============
def load_iemocap_data(dataset_path):
    """
    Load IEMOCAP by parsing EmoEvaluation annotation files.
    IEMOCAP stores emotions in separate .txt files, not in wav filenames.
    """
    files, labels = [], []
    emotion_map = {'ang': 0, 'hap': 1, 'exc': 1, 'sad': 2, 'neu': 3}

    # Dictionary to store utterance_id -> emotion mapping
    utterance_emotions = {}

    # Find and parse all EmoEvaluation files
    print("Parsing emotion annotation files...")
    eval_files_found = 0

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            # Look for evaluation files in EmoEvaluation folders
            if f.endswith('.txt') and 'EmoEvaluation' in root:
                eval_files_found += 1
                eval_path = os.path.join(root, f)

                try:
                    with open(eval_path, 'r', errors='ignore') as ef:
                        for line in ef:
                            # Pattern: [start - end] utterance_id emotion [v, a, d]
                            match = re.match(
                                r'\[[\d\.]+ - [\d\.]+\]\s+(\S+)\s+(ang|hap|exc|sad|neu|fru|sur|fea|dis|xxx|oth)',
                                line.strip()
                            )
                            if match:
                                utt_id = match.group(1)
                                emotion = match.group(2).lower()
                                if emotion in emotion_map:
                                    utterance_emotions[utt_id] = emotion_map[emotion]
                except Exception as e:
                    continue

    print(f"Found {eval_files_found} evaluation files")
    print(f"Parsed {len(utterance_emotions)} utterance labels")

    # Now find corresponding wav files
    print("Matching wav files with labels...")
    wav_files_dict = {}

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            if f.endswith('.wav'):
                # Get utterance ID (filename without extension)
                utt_id = f.replace('.wav', '')
                wav_files_dict[utt_id] = os.path.join(root, f)

    # Match wav files with emotions
    for utt_id, emotion_label in utterance_emotions.items():
        if utt_id in wav_files_dict:
            files.append(wav_files_dict[utt_id])
            labels.append(emotion_label)

    print(f"Found {len(files)} labeled audio files")

    # Class distribution
    if files:
        unique, counts = np.unique(labels, return_counts=True)
        names = ['angry', 'happy', 'sad', 'neutral']
        print("\nClass distribution:")
        for u, c in zip(unique, counts):
            print(f"  {names[u]}: {c}")

    return files, labels

audio_files, labels = load_iemocap_data(dataset_path)

if len(audio_files) == 0:
    raise ValueError("No labeled files found! Check dataset structure.")

# ============== CELL 6: Dataset Class with MFCC Feature Extraction ==============
class IEMOCAPLSTMDataset(Dataset):
    def __init__(self, audio_files, labels, config):
        self.audio_files = audio_files
        self.labels = labels
        self.config = config
        self.max_len = config.SAMPLE_RATE * config.MAX_DURATION

    def __len__(self):
        return len(self.audio_files)

    def extract_features(self, audio, sr):
        """Extract MFCC features with deltas and delta-deltas"""
        # Extract MFCCs (13 coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=self.config.N_MFCC)

        # Extract delta and delta-delta
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

        # Concatenate features (13 + 13 + 13 = 39 features)
        features = np.concatenate([mfccs, mfcc_delta, mfcc_delta2], axis=0)

        return features.T  # Shape: (time_steps, features)

    def __getitem__(self, idx):
        path = self.audio_files[idx]
        label = self.labels[idx]

        try:
            # Load audio
            audio, sr = librosa.load(path, sr=self.config.SAMPLE_RATE, mono=True)
            audio = librosa.util.normalize(audio)

            # Pad/truncate to fixed length
            if len(audio) > self.max_len:
                audio = audio[:self.max_len]
            else:
                audio = np.pad(audio, (0, self.max_len - len(audio)))

            # Extract MFCC features
            features = self.extract_features(audio, sr)

            # Pad or truncate to fixed length
            if features.shape[0] < self.config.MAX_LENGTH:
                # Pad with zeros
                padding = np.zeros((self.config.MAX_LENGTH - features.shape[0], features.shape[1]))
                features = np.vstack([features, padding])
                seq_length = features.shape[0] - padding.shape[0]
            else:
                # Truncate
                features = features[:self.config.MAX_LENGTH]
                seq_length = self.config.MAX_LENGTH

            return torch.tensor(features, dtype=torch.float32), label, seq_length

        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return zeros if error
            return torch.zeros(self.config.MAX_LENGTH, self.config.N_MFCC * 3), label, 0

# ============== CELL 7: Create DataLoaders ==============
X_train, X_test, y_train, y_test = train_test_split(
    audio_files, labels, test_size=0.2, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)

train_dataset = IEMOCAPLSTMDataset(X_train, y_train, config)
val_dataset = IEMOCAPLSTMDataset(X_val, y_val, config)
test_dataset = IEMOCAPLSTMDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
                        shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                         shuffle=False, num_workers=2)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# ============== CELL 8: ArcFace Loss Implementation ==============
class ArcFaceLoss(nn.Module):
    """
    ArcFace (Additive Angular Margin Loss) for deep face recognition

    ArcFace enhances discriminative power by adding an angular margin to the arc
    between the feature and its corresponding class weight. This forces the model
    to learn more discriminative features with better inter-class separation.

    Formula: L = -log(exp(s * cos(θ_yi + m)) / (exp(s * cos(θ_yi + m)) + Σ exp(s * cos(θ_j))))

    where:
    - s: scale parameter (typically 30-64), controls the sharpness of decision boundaries
    - m: angular margin (typically 0.3-0.5 radians), increases inter-class separation
    - θ_yi: angle between feature and target class weight
    - θ_j: angle between feature and other class weights

    The key insight: By adding margin 'm' to the target angle, we make it harder
    for the model to classify correctly, forcing it to learn more robust features.

    Args:
        in_features: Size of input feature embeddings
        out_features: Number of output classes
        s: Scale parameter (default: 30.0)
        m: Angular margin in radians (default: 0.50)
        easy_margin: If True, use easy margin (not recommended)
    """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcFaceLoss, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.easy_margin = easy_margin

        # Weight matrix: each row is a class center (normalized)
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        # Precompute cos(m) and sin(m) for efficiency
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)  # Threshold for easy_margin
        self.mm = math.sin(math.pi - m) * m

    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: Feature embeddings (batch_size, in_features)
            labels: Ground truth labels (batch_size,)
        Returns:
            loss: ArcFace loss value
        """
        # Normalize embeddings and weights
        embeddings = F.normalize(embeddings, p=2, dim=1)
        weight_norm = F.normalize(self.weight, p=2, dim=1)

        # Compute cosine similarity (logits before margin)
        cosine = F.linear(embeddings, weight_norm)
        cosine = cosine.clamp(-1, 1)  # For numerical stability

        # Compute angle θ between embedding and weight
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))

        # Compute cos(θ + m) = cos(θ)cos(m) - sin(θ)sin(m)
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        # Create one-hot encoding of labels
        one_hot = torch.zeros(cosine.size(), device=embeddings.device)
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1)

        # Apply the margin only to the target class
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)

        # Scale the output
        output *= self.s

        # Compute cross-entropy loss
        loss = F.cross_entropy(output, labels)

        return loss

# ============== CELL 9: LSTM Model with ArcFace ==============
class EmotionLSTMArcFace(nn.Module):
    """Bidirectional LSTM with ArcFace Loss for Speech Emotion Recognition"""

    def __init__(self, config):
        super(EmotionLSTMArcFace, self).__init__()

        self.hidden_size = config.HIDDEN_SIZE
        self.num_layers = config.NUM_LAYERS
        self.bidirectional = config.BIDIRECTIONAL

        input_size = config.N_MFCC * 3  # 13 MFCCs + 13 deltas + 13 delta-deltas = 39

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT if config.NUM_LAYERS > 1 else 0,
            bidirectional=config.BIDIRECTIONAL
        )

        # Embedding layers
        lstm_output_size = config.HIDDEN_SIZE * 2 if config.BIDIRECTIONAL else config.HIDDEN_SIZE
        self.fc1 = nn.Linear(lstm_output_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(config.DROPOUT)

        # Embedding layer (normalized feature space for ArcFace)
        self.embedding = nn.Linear(128, config.EMBEDDING_SIZE)
        self.bn2 = nn.BatchNorm1d(config.EMBEDDING_SIZE)

        self.relu = nn.ReLU()

    def forward(self, x, seq_lengths=None):
        """
        Args:
            x: Input features (batch, seq_len, input_size)
            seq_lengths: Actual sequence lengths (optional)
        Returns:
            embeddings: Normalized embeddings for ArcFace (batch, embedding_size)
        """
        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Use the last output (for bidirectional, concatenate both directions)
        if self.bidirectional:
            last_output = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            last_output = hidden[-1]

        # Feature extraction
        out = self.fc1(last_output)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)

        # Generate embeddings
        embeddings = self.embedding(out)
        embeddings = self.bn2(embeddings)

        return embeddings

# ============== CELL 10: Training Function ==============
def train_lstm_arcface(model, arcface_loss, train_loader, val_loader, config):
    """Train LSTM with ArcFace Loss"""

    print(f"\n{'='*60}")
    print(f"Training Bidirectional LSTM for Emotion Recognition")
    print(f"Loss Function: ArcFace (Additive Angular Margin)")
    print(f"{'='*60}")

    # Optimizer for both model and ArcFace layer
    optimizer = torch.optim.Adam(
        list(model.parameters()) + list(arcface_loss.parameters()),
        lr=config.LEARNING_RATE
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.NUM_EPOCHS)

    best_val_acc = 0
    best_model_state = None
    best_arcface_state = None
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    patience = 10
    patience_counter = 0

    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        arcface_loss.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]')
        for features, labels, seq_lengths in train_bar:
            features = features.to(config.DEVICE)
            labels = labels.to(config.DEVICE).long()

            # Forward pass
            optimizer.zero_grad()
            embeddings = model(features)
            loss = arcface_loss(embeddings, labels)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(arcface_loss.parameters(), 1.0)
            optimizer.step()

            # Statistics (for accuracy, we need to compute logits)
            with torch.no_grad():
                embeddings_norm = F.normalize(embeddings, p=2, dim=1)
                weight_norm = F.normalize(arcface_loss.weight, p=2, dim=1)
                logits = F.linear(embeddings_norm, weight_norm) * arcface_loss.s
                _, predicted = torch.max(logits, 1)
                train_correct += (predicted == labels).sum().item()
                train_total += labels.size(0)

            train_loss += loss.item()

            train_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total

        # Validation phase
        model.eval()
        arcface_loss.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for features, labels, seq_lengths in val_loader:
                features = features.to(config.DEVICE)
                labels = labels.to(config.DEVICE).long()

                embeddings = model(features)
                loss = arcface_loss(embeddings, labels)

                # Compute predictions
                embeddings_norm = F.normalize(embeddings, p=2, dim=1)
                weight_norm = F.normalize(arcface_loss.weight, p=2, dim=1)
                logits = F.linear(embeddings_norm, weight_norm) * arcface_loss.s
                _, predicted = torch.max(logits, 1)

                val_loss += loss.item()
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total

        # Store metrics
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

        scheduler.step()

        # Early stopping
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            best_arcface_state = {k: v.cpu().clone() for k, v in arcface_loss.state_dict().items()}
            print('  ✅ Best model saved!')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'\n⚠️ Early stopping triggered after {epoch+1} epochs')
                break

    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)
        arcface_loss.load_state_dict(best_arcface_state)

    return model, arcface_loss, history, best_val_acc

# ============== CELL 11: Evaluation Function ==============
def evaluate_model_arcface(model, arcface_loss, test_loader, config):
    """Evaluate model on test set"""
    model.eval()
    arcface_loss.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for features, labels, seq_lengths in test_loader:
            features = features.to(config.DEVICE)

            # Get embeddings
            embeddings = model(features)

            # Compute logits for prediction
            embeddings_norm = F.normalize(embeddings, p=2, dim=1)
            weight_norm = F.normalize(arcface_loss.weight, p=2, dim=1)
            logits = F.linear(embeddings_norm, weight_norm) * arcface_loss.s

            preds.extend(logits.argmax(1).cpu().numpy())
            true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')
    cm = confusion_matrix(true_labels, preds)

    return acc, f1, cm, preds, true_labels

# ============== CELL 12: Train and Evaluate ==============
print(f"\n{'='*60}")
print("Initializing LSTM Model with ArcFace")
print(f"{'='*60}")

model = EmotionLSTMArcFace(config).to(config.DEVICE)
arcface_loss = ArcFaceLoss(
    in_features=config.EMBEDDING_SIZE,
    out_features=config.NUM_CLASSES,
    s=config.ARCFACE_S,
    m=config.ARCFACE_M
).to(config.DEVICE)

print(f"\n📊 Model Architecture:")
print(model)
print(f"\nTotal model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"ArcFace layer parameters: {sum(p.numel() for p in arcface_loss.parameters()):,}")
print(f"\n🔧 Loss Function: ArcFace (Additive Angular Margin Loss)")
print(f"   Scale (s): {config.ARCFACE_S}")
print(f"   Margin (m): {config.ARCFACE_M} radians")
print(f"   Embedding size: {config.EMBEDDING_SIZE}")

# Train
model, arcface_loss, history, best_val_acc = train_lstm_arcface(
    model, arcface_loss, train_loader, val_loader, config
)

# Evaluate on test set
print(f"\n{'='*60}")
print("Evaluating on Test Set")
print(f"{'='*60}")

test_acc, test_f1, cm, preds, true_labels = evaluate_model_arcface(
    model, arcface_loss, test_loader, config
)

print(f"\n🏆 FINAL RESULTS:")
print(f"  Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"  Test Accuracy: {test_acc*100:.2f}%")
print(f"  Test F1 Score: {test_f1:.4f}")

# Classification report
emotion_names = ['Angry', 'Happy', 'Sad', 'Neutral']
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, preds, target_names=emotion_names))

# ============== CELL 13: Visualization ==============
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Training Loss
ax1 = axes[0, 0]
ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title(f'Training and Validation Loss\n(ArcFace: s={config.ARCFACE_S}, m={config.ARCFACE_M})',
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Training Accuracy
ax2 = axes[0, 1]
ax2.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title(f'Training and Validation Accuracy\n(ArcFace: s={config.ARCFACE_S}, m={config.ARCFACE_M})',
              fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: Confusion Matrix
ax3 = axes[1, 0]
im = ax3.imshow(cm, cmap='Blues', aspect='auto')
ax3.set_xticks(range(4))
ax3.set_yticks(range(4))
ax3.set_xticklabels(emotion_names, rotation=45)
ax3.set_yticklabels(emotion_names)
ax3.set_xlabel('Predicted', fontsize=12)
ax3.set_ylabel('True', fontsize=12)
ax3.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
for i in range(4):
    for j in range(4):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax3.text(j, i, cm[i, j], ha='center', va='center',
                fontsize=14, color=text_color, fontweight='bold')
plt.colorbar(im, ax=ax3)

# Plot 4: Per-class Accuracy
ax4 = axes[1, 1]
per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
bars = ax4.bar(emotion_names, per_class_acc, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'])
ax4.set_ylabel('Accuracy (%)', fontsize=12)
ax4.set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
ax4.set_ylim([0, 100])
for bar, val in zip(bars, per_class_acc):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.1f}%', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('lstm_emotion_results_arcface.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Results saved to 'lstm_emotion_results_arcface.png'")

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'arcface_state_dict': arcface_loss.state_dict(),
    'config': config,
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'history': history,
    'loss_function': 'ArcFace',
    'arcface_s': config.ARCFACE_S,
    'arcface_m': config.ARCFACE_M
}, 'lstm_emotion_model_arcface.pth')

print("✅ Model saved to 'lstm_emotion_model_arcface.pth'")

In [ ]:
from google.colab import files
files.download('lstm_emotion_model_arcface.pth')
print("✅ Model downloaded as 'lstm_emotion_model_arcface.pth")

In [ ]:
"""
LSTM for Speech Emotion Recognition on IEMOCAP
Loss Function: Concordance Correlation Coefficient (CCC) Loss
"""

# ============== CELL 1: Installation ==============
# !pip install kagglehub librosa soundfile matplotlib tqdm numpy pandas scikit-learn

# ============== CELL 2: Download & Explore Dataset ==============
import kagglehub
import os

print("Downloading IEMOCAP dataset...")
dataset_path = kagglehub.dataset_download("dejolilandry/iemocapfullrelease")
print(f"Dataset path: {dataset_path}")

# Explore structure
print("\nExploring dataset structure...")
for item in os.listdir(dataset_path)[:10]:
    print(f"  {item}")

# Find wav files
wav_files = []
for root, dirs, files in os.walk(dataset_path):
    for f in files:
        if f.endswith('.wav'):
            wav_files.append(os.path.join(root, f))
print(f"\nTotal .wav files: {len(wav_files)}")
if wav_files:
    print("Sample paths:")
    for p in wav_files[:3]:
        print(f"  {p}")

# ============== CELL 3: Imports ==============
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import warnings
import re
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============== CELL 4: Configuration ==============
class Config:
    SAMPLE_RATE = 16000
    MAX_DURATION = 20  # seconds
    N_MFCC = 13  # Number of MFCC coefficients
    MAX_LENGTH = 128  # Max time steps for LSTM
    BATCH_SIZE = 32
    NUM_EPOCHS = 20
    LEARNING_RATE = 0.001
    NUM_CLASSES = 4  # angry, happy, sad, neutral
    DEVICE = device
    HIDDEN_SIZE = 128
    NUM_LAYERS = 2
    DROPOUT = 0.3
    BIDIRECTIONAL = True
    # CCC Loss parameters
    CCC_LAMBDA = 1.0  # Weight for CCC loss component (balance between CCC and CE)
    USE_HYBRID_LOSS = True  # If True, combines CCC with Cross-Entropy

config = Config()

# ============== CELL 5: Data Loading ==============
def load_iemocap_data(dataset_path):
    """
    Load IEMOCAP by parsing EmoEvaluation annotation files.
    IEMOCAP stores emotions in separate .txt files, not in wav filenames.
    """
    files, labels = [], []
    emotion_map = {'ang': 0, 'hap': 1, 'exc': 1, 'sad': 2, 'neu': 3}

    # Dictionary to store utterance_id -> emotion mapping
    utterance_emotions = {}

    # Find and parse all EmoEvaluation files
    print("Parsing emotion annotation files...")
    eval_files_found = 0

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            # Look for evaluation files in EmoEvaluation folders
            if f.endswith('.txt') and 'EmoEvaluation' in root:
                eval_files_found += 1
                eval_path = os.path.join(root, f)

                try:
                    with open(eval_path, 'r', errors='ignore') as ef:
                        for line in ef:
                            # Pattern: [start - end] utterance_id emotion [v, a, d]
                            match = re.match(
                                r'\[[\d\.]+ - [\d\.]+\]\s+(\S+)\s+(ang|hap|exc|sad|neu|fru|sur|fea|dis|xxx|oth)',
                                line.strip()
                            )
                            if match:
                                utt_id = match.group(1)
                                emotion = match.group(2).lower()
                                if emotion in emotion_map:
                                    utterance_emotions[utt_id] = emotion_map[emotion]
                except Exception as e:
                    continue

    print(f"Found {eval_files_found} evaluation files")
    print(f"Parsed {len(utterance_emotions)} utterance labels")

    # Now find corresponding wav files
    print("Matching wav files with labels...")
    wav_files_dict = {}

    for root, dirs, fnames in os.walk(dataset_path):
        for f in fnames:
            if f.endswith('.wav'):
                # Get utterance ID (filename without extension)
                utt_id = f.replace('.wav', '')
                wav_files_dict[utt_id] = os.path.join(root, f)

    # Match wav files with emotions
    for utt_id, emotion_label in utterance_emotions.items():
        if utt_id in wav_files_dict:
            files.append(wav_files_dict[utt_id])
            labels.append(emotion_label)

    print(f"Found {len(files)} labeled audio files")

    # Class distribution
    if files:
        unique, counts = np.unique(labels, return_counts=True)
        names = ['angry', 'happy', 'sad', 'neutral']
        print("\nClass distribution:")
        for u, c in zip(unique, counts):
            print(f"  {names[u]}: {c}")

    return files, labels

audio_files, labels = load_iemocap_data(dataset_path)

if len(audio_files) == 0:
    raise ValueError("No labeled files found! Check dataset structure.")

# ============== CELL 6: Dataset Class with MFCC Feature Extraction ==============
class IEMOCAPLSTMDataset(Dataset):
    def __init__(self, audio_files, labels, config):
        self.audio_files = audio_files
        self.labels = labels
        self.config = config
        self.max_len = config.SAMPLE_RATE * config.MAX_DURATION

    def __len__(self):
        return len(self.audio_files)

    def extract_features(self, audio, sr):
        """Extract MFCC features with deltas and delta-deltas"""
        # Extract MFCCs (13 coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=self.config.N_MFCC)

        # Extract delta and delta-delta
        mfcc_delta = librosa.feature.delta(mfccs)
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)

        # Concatenate features (13 + 13 + 13 = 39 features)
        features = np.concatenate([mfccs, mfcc_delta, mfcc_delta2], axis=0)

        return features.T  # Shape: (time_steps, features)

    def __getitem__(self, idx):
        path = self.audio_files[idx]
        label = self.labels[idx]

        try:
            # Load audio
            audio, sr = librosa.load(path, sr=self.config.SAMPLE_RATE, mono=True)
            audio = librosa.util.normalize(audio)

            # Pad/truncate to fixed length
            if len(audio) > self.max_len:
                audio = audio[:self.max_len]
            else:
                audio = np.pad(audio, (0, self.max_len - len(audio)))

            # Extract MFCC features
            features = self.extract_features(audio, sr)

            # Pad or truncate to fixed length
            if features.shape[0] < self.config.MAX_LENGTH:
                # Pad with zeros
                padding = np.zeros((self.config.MAX_LENGTH - features.shape[0], features.shape[1]))
                features = np.vstack([features, padding])
                seq_length = features.shape[0] - padding.shape[0]
            else:
                # Truncate
                features = features[:self.config.MAX_LENGTH]
                seq_length = self.config.MAX_LENGTH

            return torch.tensor(features, dtype=torch.float32), label, seq_length

        except Exception as e:
            print(f"Error loading {path}: {e}")
            # Return zeros if error
            return torch.zeros(self.config.MAX_LENGTH, self.config.N_MFCC * 3), label, 0

# ============== CELL 7: Create DataLoaders ==============
X_train, X_test, y_train, y_test = train_test_split(
    audio_files, labels, test_size=0.2, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)

train_dataset = IEMOCAPLSTMDataset(X_train, y_train, config)
val_dataset = IEMOCAPLSTMDataset(X_val, y_val, config)
test_dataset = IEMOCAPLSTMDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
                          shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
                        shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE,
                         shuffle=False, num_workers=2)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# ============== CELL 8: CCC Loss Implementation ==============
class CCCLoss(nn.Module):
    """
    Concordance Correlation Coefficient (CCC) Loss

    The CCC measures the agreement between two variables by combining:
    1. Pearson correlation (measures linear relationship)
    2. Deviation from the 45-degree line (measures accuracy)

    Originally designed for continuous emotion recognition (valence/arousal),
    but can be adapted for classification by treating predictions as continuous values.

    CCC Formula:
    CCC = (2 * ρ * σ_x * σ_y) / (σ_x² + σ_y² + (μ_x - μ_y)²)

    where:
    - ρ: Pearson correlation between predictions and targets
    - σ_x, σ_y: Standard deviations of predictions and targets
    - μ_x, μ_y: Means of predictions and targets

    For classification, we convert categorical labels to continuous representations.
    Loss = 1 - CCC (to minimize, since CCC ∈ [-1, 1], higher is better)

    Args:
        eps: Small constant for numerical stability
        reduction: 'mean', 'sum', or 'none'
    """
    def __init__(self, eps=1e-8, reduction='mean'):
        super(CCCLoss, self).__init__()
        self.eps = eps
        self.reduction = reduction

    def forward(self, predictions, targets):
        """
        Args:
            predictions: Model outputs (batch_size, num_classes) - logits or probabilities
            targets: Ground truth labels (batch_size,) - class indices
        Returns:
            loss: 1 - CCC (to be minimized)
        """
        # Convert predictions to probabilities
        pred_probs = F.softmax(predictions, dim=1)

        # Convert categorical targets to one-hot encoding
        targets_one_hot = F.one_hot(targets, num_classes=predictions.shape[1]).float()

        # Flatten for CCC calculation
        pred_flat = pred_probs.view(-1)
        target_flat = targets_one_hot.view(-1)

        # Calculate means
        pred_mean = torch.mean(pred_flat)
        target_mean = torch.mean(target_flat)

        # Calculate variances and covariance
        pred_var = torch.var(pred_flat)
        target_var = torch.var(target_flat)
        covariance = torch.mean((pred_flat - pred_mean) * (target_flat - target_mean))

        # Calculate CCC
        numerator = 2 * covariance
        denominator = pred_var + target_var + (pred_mean - target_mean) ** 2 + self.eps

        ccc = numerator / denominator

        # Loss is 1 - CCC (since we want to maximize CCC)
        loss = 1 - ccc

        if self.reduction == 'mean':
            return loss
        elif self.reduction == 'sum':
            return loss * pred_flat.shape[0]
        else:
            return loss


class HybridCCCLoss(nn.Module):
    """
    Hybrid loss combining CCC Loss and Cross-Entropy Loss

    This combines:
    1. CCC Loss: Measures agreement between predicted and true distributions
    2. Cross-Entropy: Standard classification loss for discrete categories

    The hybrid approach provides:
    - Strong classification performance (via CE)
    - Better correlation with ground truth (via CCC)
    - More robust training

    Loss = λ * CCC_Loss + (1 - λ) * CE_Loss

    Args:
        lambda_ccc: Weight for CCC loss component (0 to 1)
        eps: Small constant for numerical stability
    """
    def __init__(self, lambda_ccc=0.5, eps=1e-8):
        super(HybridCCCLoss, self).__init__()
        self.lambda_ccc = lambda_ccc
        self.ccc_loss = CCCLoss(eps=eps)
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, predictions, targets):
        """
        Args:
            predictions: Model outputs (batch_size, num_classes)
            targets: Ground truth labels (batch_size,)
        Returns:
            Combined loss value
        """
        # Calculate both losses
        ccc = self.ccc_loss(predictions, targets)
        ce = self.ce_loss(predictions, targets)

        # Combine with lambda weighting
        loss = self.lambda_ccc * ccc + (1 - self.lambda_ccc) * ce

        return loss, ccc, ce


# ============== CELL 9: LSTM Model ==============
class EmotionLSTM(nn.Module):
    """Bidirectional LSTM for Speech Emotion Recognition"""

    def __init__(self, config):
        super(EmotionLSTM, self).__init__()

        self.hidden_size = config.HIDDEN_SIZE
        self.num_layers = config.NUM_LAYERS
        self.bidirectional = config.BIDIRECTIONAL

        input_size = config.N_MFCC * 3  # 13 MFCCs + 13 deltas + 13 delta-deltas = 39

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT if config.NUM_LAYERS > 1 else 0,
            bidirectional=config.BIDIRECTIONAL
        )

        # Fully connected layers
        lstm_output_size = config.HIDDEN_SIZE * 2 if config.BIDIRECTIONAL else config.HIDDEN_SIZE
        self.fc1 = nn.Linear(lstm_output_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout = nn.Dropout(config.DROPOUT)
        self.fc2 = nn.Linear(64, config.NUM_CLASSES)
        self.relu = nn.ReLU()

    def forward(self, x, seq_lengths=None):
        # x shape: (batch, seq_len, input_size)

        # LSTM forward pass
        lstm_out, (hidden, cell) = self.lstm(x)

        # Use the last output (for bidirectional, concatenate both directions)
        if self.bidirectional:
            # Concatenate the final forward and backward hidden states
            last_output = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            last_output = hidden[-1]

        # Fully connected layers
        out = self.fc1(last_output)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return out

# ============== CELL 10: Training Function ==============
def train_lstm(model, train_loader, val_loader, config):
    """Train LSTM with CCC Loss"""

    print(f"\n{'='*60}")
    print(f"Training Bidirectional LSTM for Emotion Recognition")
    if config.USE_HYBRID_LOSS:
        print(f"Loss Function: Hybrid CCC + Cross-Entropy (λ={config.CCC_LAMBDA})")
    else:
        print(f"Loss Function: CCC Loss Only")
    print(f"{'='*60}")

    # Use CCC Loss (Hybrid or Pure)
    if config.USE_HYBRID_LOSS:
        criterion = HybridCCCLoss(lambda_ccc=config.CCC_LAMBDA)
    else:
        criterion = CCCLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.NUM_EPOCHS)

    best_val_acc = 0
    best_model_state = None
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'train_ccc': [], 'train_ce': [],
        'val_ccc': [], 'val_ce': []
    }
    patience = 10
    patience_counter = 0

    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        train_ccc_total = 0.0
        train_ce_total = 0.0

        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]')
        for features, labels, seq_lengths in train_bar:
            features = features.to(config.DEVICE)
            labels = labels.to(config.DEVICE).long()

            # Forward pass
            optimizer.zero_grad()
            outputs = model(features)

            if config.USE_HYBRID_LOSS:
                loss, ccc_loss, ce_loss = criterion(outputs, labels)
                train_ccc_total += ccc_loss.item()
                train_ce_total += ce_loss.item()
            else:
                loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # Statistics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            train_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        avg_train_ccc = train_ccc_total / len(train_loader) if config.USE_HYBRID_LOSS else 0
        avg_train_ce = train_ce_total / len(train_loader) if config.USE_HYBRID_LOSS else 0

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_ccc_total = 0.0
        val_ce_total = 0.0

        with torch.no_grad():
            for features, labels, seq_lengths in val_loader:
                features = features.to(config.DEVICE)
                labels = labels.to(config.DEVICE).long()

                outputs = model(features)

                if config.USE_HYBRID_LOSS:
                    loss, ccc_loss, ce_loss = criterion(outputs, labels)
                    val_ccc_total += ccc_loss.item()
                    val_ce_total += ce_loss.item()
                else:
                    loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        avg_val_ccc = val_ccc_total / len(val_loader) if config.USE_HYBRID_LOSS else 0
        avg_val_ce = val_ce_total / len(val_loader) if config.USE_HYBRID_LOSS else 0

        # Store metrics
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        history['train_ccc'].append(avg_train_ccc)
        history['train_ce'].append(avg_train_ce)
        history['val_ccc'].append(avg_val_ccc)
        history['val_ce'].append(avg_val_ce)

        print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%')
        if config.USE_HYBRID_LOSS:
            print(f'    CCC: {avg_train_ccc:.4f}, CE: {avg_train_ce:.4f}')
        print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        if config.USE_HYBRID_LOSS:
            print(f'    CCC: {avg_val_ccc:.4f}, CE: {avg_val_ce:.4f}')

        scheduler.step()

        # Early stopping
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            patience_counter = 0
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            print('  ✅ Best model saved!')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f'\n⚠️ Early stopping triggered after {epoch+1} epochs')
                break

    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    return model, history, best_val_acc

# ============== CELL 11: Evaluation Function ==============
def evaluate_model(model, test_loader, config):
    """Evaluate model on test set"""
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for features, labels, seq_lengths in test_loader:
            features = features.to(config.DEVICE)
            outputs = model(features)
            preds.extend(outputs.argmax(1).cpu().numpy())
            true_labels.extend(labels.numpy())

    acc = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')
    cm = confusion_matrix(true_labels, preds)

    return acc, f1, cm, preds, true_labels

# ============== CELL 12: Train and Evaluate ==============
print(f"\n{'='*60}")
print("Initializing LSTM Model")
print(f"{'='*60}")

model = EmotionLSTM(config).to(config.DEVICE)

print(f"\n📊 Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"\n🔧 Loss Function: {'Hybrid CCC + CE' if config.USE_HYBRID_LOSS else 'CCC Loss'}")
if config.USE_HYBRID_LOSS:
    print(f"   Lambda (λ): {config.CCC_LAMBDA} (CCC weight)")
    print(f"   1 - Lambda: {1 - config.CCC_LAMBDA} (CE weight)")

# Train
model, history, best_val_acc = train_lstm(model, train_loader, val_loader, config)

# Evaluate on test set
print(f"\n{'='*60}")
print("Evaluating on Test Set")
print(f"{'='*60}")

test_acc, test_f1, cm, preds, true_labels = evaluate_model(model, test_loader, config)

print(f"\n🏆 FINAL RESULTS:")
print(f"  Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"  Test Accuracy: {test_acc*100:.2f}%")
print(f"  Test F1 Score: {test_f1:.4f}")

# Classification report
emotion_names = ['Angry', 'Happy', 'Sad', 'Neutral']
print(f"\n📊 Classification Report:")
print(classification_report(true_labels, preds, target_names=emotion_names))

# ============== CELL 13: Visualization ==============
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Plot 1: Training Loss
ax1 = axes[0, 0]
ax1.plot(history['train_loss'], label='Train Loss', linewidth=2)
ax1.plot(history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
loss_name = f"Hybrid (λ={config.CCC_LAMBDA})" if config.USE_HYBRID_LOSS else "CCC"
ax1.set_title(f'Training and Validation Loss\n({loss_name})',
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Plot 2: Training Accuracy
ax2 = axes[0, 1]
ax2.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
ax2.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title(f'Training and Validation Accuracy\n({loss_name})',
              fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# Plot 3: CCC and CE Loss Components (if hybrid)
ax3 = axes[0, 2]
if config.USE_HYBRID_LOSS and len(history['train_ccc']) > 0:
    ax3.plot(history['train_ccc'], label='Train CCC', linewidth=2, linestyle='--')
    ax3.plot(history['val_ccc'], label='Val CCC', linewidth=2, linestyle='--')
    ax3.plot(history['train_ce'], label='Train CE', linewidth=2, linestyle='-.')
    ax3.plot(history['val_ce'], label='Val CE', linewidth=2, linestyle='-.')
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Loss', fontsize=12)
    ax3.set_title('CCC and CE Loss Components', fontsize=14, fontweight='bold')
    ax3.legend(fontsize=9)
    ax3.grid(alpha=0.3)
else:
    ax3.text(0.5, 0.5, 'Pure CCC Loss\n(No components to display)',
             ha='center', va='center', fontsize=12)
    ax3.axis('off')

# Plot 4: Confusion Matrix
ax4 = axes[1, 0]
im = ax4.imshow(cm, cmap='Blues', aspect='auto')
ax4.set_xticks(range(4))
ax4.set_yticks(range(4))
ax4.set_xticklabels(emotion_names, rotation=45)
ax4.set_yticklabels(emotion_names)
ax4.set_xlabel('Predicted', fontsize=12)
ax4.set_ylabel('True', fontsize=12)
ax4.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
for i in range(4):
    for j in range(4):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        ax4.text(j, i, cm[i, j], ha='center', va='center',
                fontsize=14, color=text_color, fontweight='bold')
plt.colorbar(im, ax=ax4)

# Plot 5: Per-class Accuracy
ax5 = axes[1, 1]
per_class_acc = cm.diagonal() / cm.sum(axis=1) * 100
bars = ax5.bar(emotion_names, per_class_acc, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'])
ax5.set_ylabel('Accuracy (%)', fontsize=12)
ax5.set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
ax5.set_ylim([0, 100])
for bar, val in zip(bars, per_class_acc):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f'{val:.1f}%', ha='center', fontsize=11, fontweight='bold')

# Plot 6: Loss comparison table
ax6 = axes[1, 2]
ax6.axis('off')
loss_info = [
    ['Metric', 'Value'],
    ['Loss Type', 'Hybrid' if config.USE_HYBRID_LOSS else 'Pure CCC'],
    ['CCC Lambda', f'{config.CCC_LAMBDA:.2f}' if config.USE_HYBRID_LOSS else 'N/A'],
    ['Best Val Acc', f'{best_val_acc:.2f}%'],
    ['Test Acc', f'{test_acc*100:.2f}%'],
    ['Test F1', f'{test_f1:.4f}']
]
table = ax6.table(cellText=loss_info, cellLoc='left', loc='center',
                  colWidths=[0.5, 0.5])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)
for i in range(len(loss_info)):
    if i == 0:
        table[(i, 0)].set_facecolor('#4CAF50')
        table[(i, 1)].set_facecolor('#4CAF50')
        table[(i, 0)].set_text_props(weight='bold', color='white')
        table[(i, 1)].set_text_props(weight='bold', color='white')
    else:
        table[(i, 0)].set_facecolor('#f0f0f0')
ax6.set_title('Training Summary', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('lstm_emotion_results_ccc_loss.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Results saved to 'lstm_emotion_results_ccc_loss.png'")

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'test_accuracy': test_acc,
    'test_f1': test_f1,
    'history': history,
    'loss_function': 'Hybrid CCC + CE' if config.USE_HYBRID_LOSS else 'CCC Loss',
    'ccc_lambda': config.CCC_LAMBDA if config.USE_HYBRID_LOSS else None
}, 'lstm_emotion_model_ccc_loss.pth')

print("✅ Model saved to 'lstm_emotion_model_ccc_loss.pth'")

In [ ]:
from google.colab import files
files.download('lstm_emotion_model_ccc_loss.pth')
print("✅ Model downloaded as 'lstm_emotion_model_ccc_loss.pth")